# Matrix calculus - differentiating the determinant

There is this matrix calculus PDF available in various places on the internet: http://www.doc.ic.ac.uk/~ahanda/referencepdfs/MatrixCalculus.pdf

It contains an example with chain rule applied to differentiation of a determinant. We thought the example had a typo in it but weren't sure. So, to help back it up, coded up an example in `sympy`. 

The example reads:

Suppose we have a matrix $Y = [y_{ij}]$ whose components are functions of a matrix $X = [x_{rs}]$, that is $y_{ij} = f_{ij}(x_{rs})$, and set out to build the matrix

$$\Large{\frac{ \partial |\mathbf{Y}|} {\partial \mathbf{X}} (D.31)}$$ 

Using the chain rule we can write

$$\Large{\frac{ \partial |\mathbf{Y}|} {\partial x_{rs}} = \sum_i \sum_j \mathbf{Y}_{ij} \frac{\partial |\mathbf{Y}|}{\partial y_{ij}} \frac{\partial y_{ij}}{\partial x_{rs}} (D.32)}$$ 

There is a typo in the above: the cofactor $\mathbf{Y}_{ij} $ should not be there, but we weren't sure about that. 

We think this equation should look like this:

$$\Large{\frac{ \partial |\mathbf{Y}|} {\partial x_{rs}} = \sum_i \sum_j  \frac{\partial |\mathbf{Y}|}{\partial y_{ij}} \frac{\partial y_{ij}}{\partial x_{rs}} (D.32A)}$$ 

So, test it out with sympy: Define  $Y = [y_{ij}]$ whose components are functions of a matrix $X = [x_{rs}]$, that is $y_{ij} = f_{ij}(x_{rs})$, and do the actual calculation.

We will use a 3x3 matrix.

$
\mathbf{X} = \begin{vmatrix} 
   x_{11} & x_{12} & x_{13}  \\
   x_{21} & x_{22} & x_{23}  \\
   x_{31} & x_{32} & x_{33}  \\
   \end{vmatrix}
$,

$
\mathbf{Y} = \begin{vmatrix} 
   {x_{11}}^2 & 4x_{12}-10 & \frac{1}{x_{13}}  \\
   \frac{1}{{x_{21}}^2} & -10x_{22} & cos(x_{23})  \\
   \sqrt{x_{31}} & exp(x_{32}) & \frac{1}{exp(x_{33})}  \\
   \end{vmatrix}
$


In [35]:
from sympy import symbols, Matrix, cos, sqrt, exp, diff, det

# Define the x and y symbols. We will express y_ij in terms of x_ij
# Then we will create X and Y matrices from the below. 
x11, x12, x13, x21, x22, x23, x31, x32, x33 = symbols("x11 x12 x13 x21 x22 x23 x31 x32 x33")
y11, y12, y13, y21, y22, y23, y31, y32, y33 = symbols("y11 y12 y13 y21 y22 y23 y31 y32 y33")

# We also need d(det(Y))/dy_ij. 
# Sympy cannot handle this differentiation in terms of y_ij if Y is expressed in terms of x_ij
# so we will have to re-create a Y matrix in terms of yy_ij, 
# where value of y_ij = value of yy_ij
yy11, yy12, yy13, yy21, yy22, yy23, yy31, yy32, yy33 = symbols("yy11 yy12 yy13 yy21 yy22 yy23 yy31 yy32 yy33")

# Now define the functions

y11 = x11**2; y12 = x12 * 4 - 10; y13 = 1/x13

y21 = 1/(x21**2); y22 = -10*x22; y23 = cos(x23)

y31 = sqrt(x31); y32 = exp(x32) ; y33 = 1/exp(x33)

# Define the variables as collections just for convenience

ys = y11, y12, y13, y21, y22, y23, y31, y32, y33

yys = yy11, yy12, yy13, yy21, yy22, yy23, yy31, yy32, yy33

# define the matrices
Y = Matrix([[y11, y12, y13], [y21, y22, y23], [y31, y32, y33]])
YY = Matrix([[yy11, yy12, yy13], [yy21, yy22, yy23], [yy31, yy32, yy33]])

# Put in values for all the xs,
xs = dict(
    x11=5, x12=-4, x13=3, 
    x21=0.5, x22=6, x23=-5, 
    x31=2, x32=0.5, x33=0.6
)

# calculate values for the yys
yy_values = {
    # in the below, eval(f"y{i}{j}}") fetches the function expression y_ij 
    # then calls .evalf on that
    f"yy{i}{j}": eval(f"y{i}{j}").evalf(subs=vars) for i in range(1, 4) for j in range(1, 4)
}

In [36]:
all_vars = yy_values.copy()
all_vars.update(xs)

Calculate the LHS of $(D32.A)$ for $x_{rs} = x_{11}$
    
$$\Large{\frac{ \partial |\mathbf{Y}|} {\partial x_{rs}}}$$

In [37]:
ddetY_dx11 = diff(det(Y), x11)

lhs_value = ddetY_dx11.evalf(subs=all_vars)

Calculate the RHS of $(D32.A)$:


$$\Large{\sum_i \sum_j  \frac{\partial |\mathbf{Y}|}{\partial y_{ij}} \frac{\partial y_{ij}}{\partial x_{rs}} }$$ 

In [27]:
def all_partials(x):
    # x: the x_ij variable required
    # yes, I know this uses stuff defined in external scope
    out = [
        diff(det(YY), yy) * diff(y, x)
        for y, yy in zip(ys, yys)
    ]
    all_partials_values = [
        expr.evalf(subs=all_vars) for expr in out
    ]
    return sum(all_partials_values)
rhs_value = all_partials(x11)

In [28]:
print(lhs_value)

-333.963780445081


In [29]:
print(rhs_value)

-333.963780445081


Just to be sure we didn't just get lucky, compute this for every $x_{ij}$

In [34]:
import numpy
varnames = "x11,x12,x13,x21,x22,x23,x31,x32,x33".split(",")

for varname in varnames:
    x = eval(varname)
    ddetY_dx = diff(det(Y), x)
    lhs_value = ddetY_dx.evalf(subs=all_vars)
    rhs_value = all_partials(x)
    assert numpy.allclose(float(lhs_value), float(rhs_value))
    print(f"{varname}: LHS={lhs_value}, RHS={rhs_value}")

x11: LHS=-333.963780445081, RHS=-333.963780445081
x12: LHS=-7.17635053824648, RHS=-7.17635053824648
x13: LHS=-10.1608554250207, RHS=-10.1608554250207
x21: LHS=-237.098820725516, RHS=-237.098820725516
x22: LHS=-132.488863815596, RHS=-132.488863815596
x23: LHS=74.7841877931474, RHS=74.7841877931474
x31: LHS=4.46353489807131, RHS=4.46353489807131
x32: LHS=-9.49370194406247, RHS=-9.49370194406247
x33: LHS=766.141043987261, RHS=766.141043987261


Suppose we have $\mathbf{Y} \in \mathbf{R^{N \times N}}$, $Z = |\mathbf{Y}|\mathbf{Y}^{-1}$: $\mathbf{Z}$ is the adjugate of $\mathbf{X}$ (transpose of cofactors)

We can check this: 

In [39]:
YY.evalf(subs=all_vars)

Matrix([
[           25.0,            -26.0, 0.333333333333333],
[            4.0,            -60.0, 0.283662185463226],
[1.4142135623731, 1.64872127070013, 0.548811636094026]])

In [44]:
YYinv = YY.inv().evalf(subs=all_vars)

In [45]:
detYY = det(YY).evalf(subs=all_vars)

In [52]:
YYadj = detYY * YYinv

In [53]:
YYadj

Matrix([
[-33.3963780445081,  14.8186762953447,  12.6247831779561],
[-1.79408763456162,  13.2488863815596, -5.75822130324732],
[ 91.4476988251862, -77.9875843892037,           -1396.0]])

In [58]:
YYcof = Matrix(
   [[YY.cofactor(row, col).evalf(subs=all_vars) for col in range(0, 3)]
    for row in range(0, 3)]
)


In [59]:
YYcof

Matrix([
[-33.3963780445081, -1.79408763456162,  91.4476988251862],
[ 14.8186762953447,  13.2488863815596, -77.9875843892037],
[ 12.6247831779561, -5.75822130324732,           -1396.0]])

$$\Large{|\mathbf{Y}| = \sum_{j} y_{ij} Y_{ij}}$$

$$\Large{\frac{\partial |\mathbf{Y}|}{\partial y_{ij}} = Y_{ij}}$$ because $\mathbf{Y_{ij}}$ does not contain any $y_{ij}$ terms and is a scalar

$$\Large{\frac{\partial |\mathbf{Y}|}{\partial \mathbf{Y}} = \mathbf{Z}^T}$$ since $\mathbf{Z}^T$ is the transpose of the cofactor matrix.